In [110]:
from gurobipy import *
import numpy as np

model = Model("mip1")

s = np.array([50, 75])
d = np.array([75, 50])

x = model.addMVar((2, 3), vtype='I', name='x')
y = model.addMVar((3, 2), vtype='I', name='y')
z = model.addMVar((2, 2), vtype='I', name='z')
p = model.addMVar((3), vtype='B', name='p')

model.update()

c = np.array([[1, 2, 8],
            [6, 3, 1]])
r = np.array([[4, 6],
            [3, 4],
            [5, 3]])
w = np.array([[4, 8],
            [7, 6]])
e = np.array([10000000, 60, 70])
m = np.array([50, 60, 68])

obj1 = quicksum(c[i,k]*x[i,k]*p[k] for i in range(2) for k in range(3))
obj2 = quicksum(r[k,j]*y[k,j]*p[k] for k in range(3) for j in range(2))
obj3 = quicksum(w[i,j]*z[i,j] for i in range(2) for j in range(2))
obj4 = quicksum(m[k]*p[k] for k in range(3))
obj = obj1 + obj2 + obj3 + obj4
model.setObjective(obj,GRB.MINIMIZE)


model.addConstr(quicksum(x[0,k]*p[k] for k in range(3)) + quicksum(z[0,j] for j in range(2)) <= s[0], name='c0')
model.addConstr(quicksum(x[1,k]*p[k] for k in range(3)) + quicksum(z[1,j] for j in range(2)) <= s[1], name='c1')
model.addConstr(quicksum(y[k,0]*p[k] for k in range(3)) + quicksum(z[i,0] for i in range(2)) >= d[0], name='c2')
model.addConstr(quicksum(y[k,1]*p[k] for k in range(3)) + quicksum(z[i,1] for i in range(2)) >= d[1], name='c3')
model.addConstr(quicksum(p[k] for k in range(3)) <= 1, name='c4')
model.addConstr(quicksum(x[i,1] for i in range(2))*p[1] <= e[1], name='c5')
model.addConstr(quicksum(x[i,2] for i in range(2))*p[2] <= e[2], name='c6')
model.addConstr(quicksum(x[i,0] for i in range(2))*p[0] >= quicksum(y[0,j] for j in range(2))*p[0], name='c7')
model.addConstr(quicksum(x[i,1] for i in range(2))*p[1] >= quicksum(y[1,j] for j in range(2))*p[1], name='c8')
model.addConstr(quicksum(x[i,2] for i in range(2))*p[2] >= quicksum(y[2,j] for j in range(2))*p[2], name='c9')

model.optimize()
print(model)
print (model.display())


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 19 columns and 3 nonzeros
Model fingerprint: 0x4bfc209c
Model has 12 quadratic objective terms
Model has 9 quadratic constraints
Variable types: 0 continuous, 19 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [4e+00, 7e+01]
  QObjective range [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [5e+01, 8e+01]
Presolve time: 0.00s
Presolved: 34 rows, 55 columns, 99 nonzeros
Presolved model has 24 SOS constraint(s)
Variable types: 0 continuous, 55 integer (15 binary)
Found heuristic solution: objective 850.0000000
Found heuristic solution: objective 675.0000000

Root relaxation: objective 6.230000e+02, 15 iterations, 0.00 seconds (0.0

In [111]:
for v in model.getVars():
    print('%s: %d' % (v.varName, v.x))
print('Obj: %f' % model.objVal)

x[0,0]: 0
x[0,1]: 0
x[0,2]: 0
x[1,0]: 0
x[1,1]: 0
x[1,2]: 70
y[0,0]: 0
y[0,1]: 0
y[1,0]: 0
y[1,1]: 0
y[2,0]: 20
y[2,1]: 50
z[0,0]: 50
z[0,1]: 0
z[1,0]: 5
z[1,1]: 0
p[0]: 0
p[1]: 0
p[2]: 1
Obj: 623.000000


In [112]:
for v in model.getVars():
    var = v.varName
    if len(var)>=6:
        index1 = int(var[2])+1
        index2 = int(var[4])+1
        name = var[0] + str(index1) + str(index2)
        print('%s: %d' % (name, v.x))
    else:
        index1 = int(var[2])+1
        name = var[0] + str(index1)
        print('%s: %d' % (name, v.x))
print('Obj: %.2d' % model.objVal)

x11: 0
x12: 0
x13: 0
x21: 0
x22: 0
x23: 70
y11: 0
y12: 0
y21: 0
y22: 0
y31: 20
y32: 50
z11: 50
z12: 0
z21: 5
z22: 0
p1: 0
p2: 0
p3: 1
Obj: 623
